In [1]:
!pip install yolov8 -q
!pip install tqdm
!pip install fastapi
!pip install uvicorn
!pip install flask
!pip install ultralytics
!pip install opencv-python-headless
!pip install python-multipart
!pip install torch torchvision torchaudio
!pip install requests
!apt-get install jq
!pip install pyngrok
!pip install flask_ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 

In [ ]:
#!sudo rm -f /usr/local/bin/ngrok
#!sudo rm -f /usr/bin/ngrok

In [2]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip -O ngrok.zip
!unzip ngrok.zip
!sudo mv ngrok /usr/local/bin/

--2024-06-14 04:53:30--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9019411 (8.6M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]   8.60M  4.71MB/s    in 1.8s    

2024-06-14 04:53:33 (4.71 MB/s) - ‘ngrok.zip’ saved [9019411/9019411]

Archive:  ngrok.zip
  inflating: ngrok                   


In [3]:
!ngrok version

ngrok version 3.11.0


In [4]:
from flask import Flask, request, send_file
from flask_ngrok import run_with_ngrok
from threading import Thread
import subprocess
import cv2
import os
import csv
from datetime import datetime
from collections import defaultdict
from copy import deepcopy
from ultralytics import YOLO
import requests
import time
from pyngrok import ngrok

# ngrok 인증 토큰 설정
authtoken = "2hVJXCKSqQraJv65bFACCo5I75J_4LXEAMxm8kNxYGAVgJUQV"
ngrok.set_auth_token(authtoken)

In [5]:
# ngrok을 통해 포트 5000을 연결
public_url = ngrok.connect(5000,hostname='romantic-goshawk-comic.ngrok-free.app').public_url  # Use a different port to avoid conflict
print(public_url)

https://romantic-goshawk-comic.ngrok-free.app


In [ ]:
from flask import Flask, request, send_file
from flask_ngrok import run_with_ngrok
import cv2
import csv
from datetime import datetime
from collections import defaultdict
from copy import deepcopy
from ultralytics import YOLO
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
run_with_ngrok(app)  # 이는 flask_ngrok에서 사용하는 것입니다.

header = [
    "Filename", "Frame", "ID", "X", "Y", "Width", "Height",
    "Keypoint_0", "Keypoint_1", "Keypoint_2", "Keypoint_3", "Keypoint_4", "Keypoint_5",
    "Keypoint_6", "Keypoint_7", "Keypoint_8", "Keypoint_9", "Keypoint_10", "Keypoint_11",
    "Keypoint_12", "Keypoint_13", "Keypoint_14", "Keypoint_15", "Keypoint_16", "Keypoint_17",
    "Keypoint_18", "Keypoint_19", "Keypoint_20", "Keypoint_21", "Keypoint_22", "Keypoint_23",
    "Keypoint_24", "Keypoint_25", "Keypoint_26", "Keypoint_27", "Keypoint_28", "Keypoint_29",
    "Keypoint_30", "Keypoint_31", "Keypoint_32", "Keypoint_33",
]

model = YOLO("yolov8n-pose.pt")
standard_width = 320
standard_height = 240

@app.route('/process-video/', methods=['POST'])
def process_video():
    logger.info("start")
    file = request.files['file']
    file_path = "uploaded_video.mp4"
    file.save(file_path)

    output_file_path = "output.csv"
    cap = cv2.VideoCapture(file_path)
    frame_count = 0
    id_count = 0
    track_history = defaultdict(lambda: [])

    logger.info("create csv...........")
    with open(output_file_path, "w") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        writer.writerow(header)

        logger.info("feature extract")
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            frame_count += 1
            frame = cv2.resize(frame, (standard_width, standard_height))
            results = model.track(frame, persist=True, verbose=False)
            if results[0].boxes and results[0].boxes.id:
                track_ids = results[0].boxes.id.int().cpu().tolist()
                for i, box in zip(range(len(track_ids)), results[0].boxes.xywhn.cpu()):
                    keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                    box_list = box.numpy().flatten().tolist()
                    box_and_keypoints = box_list + keypoints
                    track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])

        for key in track_history.keys():
            for f_count, b_and_k in track_history[key]:
                row = ["uploaded_video.mp4"] + f_count + [id_count + key] + b_and_k
                writer.writerow(row)

        id_count += len(track_history.keys())
        cap.release()

    return send_file(output_file_path, mimetype='text/csv', as_attachment=True, download_name="output.csv")

if __name__ == '__main__':
    app.run()


RuntimeError: Error(s) in loading state_dict for LSTMAutoencoder:
	Missing key(s) in state_dict: "encoder.weight_ih_l1", "encoder.weight_hh_l1", "encoder.bias_ih_l1", "encoder.bias_hh_l1", "decoder.weight_ih_l1", "decoder.weight_hh_l1", "decoder.bias_ih_l1", "decoder.bias_hh_l1". 
	Unexpected key(s) in state_dict: "encoder2.weight_ih_l0", "encoder2.weight_hh_l0", "encoder2.bias_ih_l0", "encoder2.bias_hh_l0", "decoder2.weight_ih_l0", "decoder2.weight_hh_l0", "decoder2.bias_ih_l0", "decoder2.bias_hh_l0". 
	size mismatch for encoder.weight_ih_l0: copying a param with shape torch.Size([400, 38]) from checkpoint, the shape in current model is torch.Size([512, 38]).
	size mismatch for encoder.weight_hh_l0: copying a param with shape torch.Size([400, 100]) from checkpoint, the shape in current model is torch.Size([512, 128]).
	size mismatch for encoder.bias_ih_l0: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for 

In [ ]:
# ngrok을 통해 포트 5000을 연결
public_url = ngrok.connect(5000,hostname='romantic-goshawk-comic.ngrok-free.app').public_url  # Use a different port to avoid conflict

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import cv2
import csv
import torch
import torch.nn as nn
import os
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict
from copy import deepcopy
from ultralytics import YOLO
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
run_with_ngrok(app)  # 이는 flask_ngrok에서 사용하는 것입니다.

header = [
    "Filename", "Frame", "ID", "X", "Y", "Width", "Height",
    "Keypoint_0", "Keypoint_1", "Keypoint_2", "Keypoint_3", "Keypoint_4", "Keypoint_5",
    "Keypoint_6", "Keypoint_7", "Keypoint_8", "Keypoint_9", "Keypoint_10", "Keypoint_11",
    "Keypoint_12", "Keypoint_13", "Keypoint_14", "Keypoint_15", "Keypoint_16", "Keypoint_17",
    "Keypoint_18", "Keypoint_19", "Keypoint_20", "Keypoint_21", "Keypoint_22", "Keypoint_23",
    "Keypoint_24", "Keypoint_25", "Keypoint_26", "Keypoint_27", "Keypoint_28", "Keypoint_29",
    "Keypoint_30", "Keypoint_31", "Keypoint_32", "Keypoint_33",
]

model = YOLO("yolov8n-pose.pt")
standard_width = 320
standard_height = 240

# 학습된 모델 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, sequence_length, n_features, prediction_time):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = nn.LSTM(
            input_size=n_features, hidden_size=100, num_layers=1, batch_first=True
        )
        self.encoder2 = nn.LSTM(
            input_size=100, hidden_size=50, num_layers=1, batch_first=True
        )
        self.decoder = nn.LSTM(
            input_size=50, hidden_size=100, num_layers=1, batch_first=True
        )
        self.decoder2 = nn.LSTM(
            input_size=100, hidden_size=n_features, num_layers=1, batch_first=True
        )

    def forward(self, x):
        x, _ = self.encoder(x)
        x, _ = self.encoder2(x)
        x, _ = self.decoder(x)
        x, _ = self.decoder2(x)
        return x

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sequence_length = 20  # 모델 학습 시 사용된 시퀀스 길이
n_features = 38  # 모델 학습 시 사용된 특징 수
prediction_time = 1  # 모델 학습 시 사용된 예측 시간

trained_model = LSTMAutoencoder(sequence_length, n_features, prediction_time)
checkpoint = torch.load("/content/drive/MyDrive/taba/준형/LSTM_20240610_005340_best.pth", map_location=device)

# # 체크포인트의 키와 값을 출력 (디버깅 목적)
# for key, value in checkpoint.items():
#     print(f"{key}: {type(value)}")

# # 필요시 세부 정보도 출력
# for key, value in checkpoint.items():
#     if isinstance(value, dict):
#         print(f"{key}:")
#         for subkey, subvalue in value.items():
#             print(f"  {subkey}: {type(subvalue)}")
#     elif isinstance(value, torch.Tensor):
#         print(f"{key}: Tensor of shape {value.shape}")
#     else:
#         print(f"{key}: {value}")

trained_model.load_state_dict(checkpoint["model_state_dict"])
trained_model.to(device)
trained_model.eval()

# 임계값 설정 (이상치 탐지를 위한)
thr = 0.3

def detect_anomalies(data, sequence_length):
    # 데이터 전처리 및 모델 예측
    data_tensor = torch.tensor(data, dtype=torch.float32).to(device)
    with torch.no_grad():
        predictions = trained_model(data_tensor)
    # 이상치 탐지 로직
    loss_fn = nn.MSELoss(reduction="none")
    loss = loss_fn(predictions, data_tensor)
    anomalies = loss.mean(dim=2) > thr  # threshold 값을 넘으면 이상치로 간주

    # 이상치가 발생한 시퀀스의 시작 프레임 인덱스 반환
    anomaly_indices = np.where(anomalies.cpu().numpy())[0]  # 이상치가 발생한 시퀀스 인덱스
    anomaly_frames = anomaly_indices * sequence_length  # 시퀀스 인덱스를 프레임 인덱스로 변환

    # 중복된 프레임 번호 제거 및 이상치가 연속된 경우 단일 시작 프레임만 반환
    unique_anomaly_frames = sorted(set(anomaly_frames))
    unique_anomaly_frames = [int(frame) for frame in unique_anomaly_frames]  # int64를 int로 변환
    return unique_anomaly_frames





@app.route('/process-video/', methods=['POST'])
def process_video():
    logger.info("start")
    file = request.files['file']
    file_path = "uploaded_video.mp4"
    file.save(file_path)

    output_file_path = "output.csv"
    cap = cv2.VideoCapture(file_path)
    frame_count = 0
    id_count = 0
    track_history = defaultdict(lambda: [])

    logger.info("create csv...........")
    with open(output_file_path, "w") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        writer.writerow(header)

        logger.info("feature extract")
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            frame_count += 1
            frame = cv2.resize(frame, (standard_width, standard_height))
            results = model.track(frame, persist=True, verbose=False)
            if results[0].boxes and results[0].boxes.id:
                track_ids = results[0].boxes.id.int().cpu().tolist()
                for i, box in zip(range(len(track_ids)), results[0].boxes.xywhn.cpu()):
                    keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                    box_list = box.numpy().flatten().tolist()
                    box_and_keypoints = box_list + keypoints
                    track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])

        for key in track_history.keys():
            for f_count, b_and_k in track_history[key]:
                row = ["uploaded_video.mp4"] + f_count + [id_count + key] + b_and_k
                writer.writerow(row)

        id_count += len(track_history.keys())
        cap.release()

    # CSV 파일 로드
    df = pd.read_csv(output_file_path)

    # 필요한 데이터 추출 및 전처리
    data = df.iloc[:, 8:].values  # Keypoint 데이터 추출

    # 데이터 크기 확인
    total_data_points = data.shape[0]
    logger.info(f"Total data points: {total_data_points}")

    # 패딩을 사용하여 데이터 크기를 맞춤
    if total_data_points % (sequence_length * n_features) != 0:
        pad_length = (sequence_length * n_features) - (total_data_points % (sequence_length * n_features))
        data = np.pad(data, ((0, pad_length), (0, 0)), mode='constant')
        logger.info(f"Padded data points: {data.shape[0]}")

    data = data.reshape(-1, sequence_length, n_features)

    # 이상행동 탐지
    anomaly_frames = detect_anomalies(data, sequence_length)

    # 결과 반환
    return jsonify({"anomaly_frames": anomaly_frames})

if __name__ == '__main__':
    app.run()



100%|██████████| 6.51M/6.51M [00:00<00:00, 81.9MB/s]


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://romantic-goshawk-comic.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


In [5]:
from collections import defaultdict
import cv2
import numpy as np
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

In [6]:
from torchvision import transforms
from datetime import datetime

In [ ]:
# ngrok을 통해 포트 5000을 연결
public_url = ngrok.connect(5000,hostname='romantic-goshawk-comic.ngrok-free.app').public_url  # Use a different port to avoid conflict

from flask import Flask, request, jsonify
import cv2
import torch
import torch.nn as nn
import numpy as np
from datetime import datetime
from collections import defaultdict
from copy import deepcopy
from ultralytics import YOLO
from sklearn.preprocessing import MinMaxScaler
from flask_ngrok import run_with_ngrok
import logging
import os

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
run_with_ngrok(app)  # ngrok 사용

# YOLOv8 모델 로드
model = YOLO("yolov8n-pose.pt")

# LSTM 오토인코더 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, sequence_length, n_features, prediction_time):
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.prediction_time = prediction_time

        self.encoder = nn.LSTM(input_size=n_features, hidden_size=100, batch_first=True)
        self.encoder2 = nn.LSTM(input_size=100, hidden_size=50, batch_first=True)
        self.repeat_vector = nn.Sequential(
            nn.ReplicationPad1d(padding=(0, prediction_time - 1)),
            nn.ReplicationPad1d(padding=(0, 0))
        )
        self.decoder = nn.LSTM(input_size=50, hidden_size=100, batch_first=True)
        self.decoder2 = nn.LSTM(input_size=100, hidden_size=n_features, batch_first=True)

    def forward(self, x):
        _, (x, _) = self.encoder(x)
        _, (x, _) = self.encoder2(x)
        x = self.repeat_vector(x)
        _, (x, _) = self.decoder(x)
        _, (x, _) = self.decoder2(x)
        return x

# 모델 인스턴스화 및 체크포인트 로드
sequence_length = 20
prediction_time = 1
n_features = 38
threshold = 0.02

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder_model = LSTMAutoencoder(sequence_length, n_features, prediction_time)
checkpoint = torch.load("/content/drive/MyDrive/taba/유원/model/LSTMAutoEncoder/LSTM_20240610_005340_best.pth", map_location=device)
state_dict = checkpoint['model_state_dict']
autoencoder_model.load_state_dict(state_dict)
autoencoder_model.to(device)
autoencoder_model.eval()

# 비디오 처리 및 이상 행동 탐지 함수
def process_video(file_path):
    cap = cv2.VideoCapture(file_path)
    standard_width = 640
    standard_height = 480

    track_history = defaultdict(lambda: [])
    id_buffers = defaultdict(lambda: [])

    # MSE 계산 함수
    def calculate_mse(seq1, seq2):
        return np.mean(np.power(seq1 - seq2, 2))

    anomaly_times = []
    frame_count = 0
    net_mse = 0
    avg_mse = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    anomaly_start_time = None
    output_video_writer = None

    while cap.isOpened():
        success, frame = cap.read()
        frame_count += 1

        if success:
            frame = cv2.resize(frame, (standard_width, standard_height))
            results = model.track(frame, persist=True)

            if results[0].boxes is not None:
                boxes = results[0].boxes.xywh.cpu()

                if results[0].boxes.id is not None:
                    track_ids = results[0].boxes.id.int().cpu().tolist()
                    anomaly_text = ""
                    for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                        x, y, w, h = box
                        keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()

                        id_buffers[track_ids[i]].append([float(x), float(y), float(w), float(h)] + keypoints)

                        if len(id_buffers[track_ids[i]]) >= 20:
                            buffer_array = np.array(id_buffers[track_ids[i]])
                            scaler = MinMaxScaler()
                            buffer_scaled = scaler.fit_transform(buffer_array)
                            x_pred = buffer_scaled[-sequence_length:].reshape(1, sequence_length, n_features)
                            x_pred = autoencoder_model(torch.tensor(x_pred, dtype=torch.float32).to(device))
                            x_pred_original = scaler.inverse_transform(x_pred.cpu().detach().numpy().reshape(-1, n_features))
                            mse = calculate_mse(buffer_array[-prediction_time:], x_pred_original)
                            net_mse = mse + net_mse
                            avg_mse = net_mse / frame_count

                            if mse > 1.5 * avg_mse * 0.25 + 0.75 * threshold:
                                if anomaly_start_time is None:
                                    anomaly_start_time = frame_count / fps
                                    output_video_writer = cv2.VideoWriter(
                                        f'anomaly_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4',
                                        cv2.VideoWriter_fourcc(*'mp4v'),
                                        fps,
                                        (standard_width, standard_height)
                                    )
                                anomaly_text = f"Anomaly detected for ID {track_ids[i]}"
                                logger.info(anomaly_text)
                                output_video_writer.write(frame)

                            else:
                                if anomaly_start_time is not None:
                                    anomaly_end_time = frame_count / fps
                                    anomaly_times.append((anomaly_start_time, anomaly_end_time))
                                    anomaly_start_time = None
                                    output_video_writer.release()
                                    output_video_writer = None

                            id_buffers[track_ids[i]].pop(0)
                else:
                    anomaly_text = ""
                    track_ids = []

                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)

                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

        else:
            break

    if anomaly_start_time is not None:
        anomaly_end_time = frame_count / fps
        anomaly_times.append((anomaly_start_time, anomaly_end_time))
        output_video_writer.release()

    cap.release()

    with open('anomaly_times.txt', 'w') as f:
        for start, end in anomaly_times:
            f.write(f"Anomaly from {start:.2f} seconds to {end:.2f} seconds\n")

    return anomaly_times

@app.route('/process-video/', methods=['POST'])
def upload_video():
    file = request.files['file']
    file_path = "uploaded_video.mp4"
    file.save(file_path)
    anomaly_times = process_video(file_path)
    return jsonify({"anomaly_times": anomaly_times})

if __name__ == '__main__':
    app.run()



100%|██████████| 6.51M/6.51M [00:00<00:00, 144MB/s]


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://romantic-goshawk-comic.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 8.6s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 480x640 1 person, 411.8ms
Speed: 17.9ms preprocess, 411.8ms inference, 33.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 202.2ms
Speed: 2.5ms preprocess, 202.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 209.0ms
Speed: 2.4ms preprocess, 209.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 197.7ms
Speed: 5.1ms preprocess, 197.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 279.6ms
Speed: 2.3ms prepro

INFO:werkzeug:127.0.0.1 - - [14/Jun/2024 05:19:55] "POST /process-video/ HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [14/Jun/2024 05:23:19] "POST /process-video/ HTTP/1.1" 200 -



0: 480x640 1 person, 264.6ms
Speed: 2.5ms preprocess, 264.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 211.6ms
Speed: 2.8ms preprocess, 211.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 207.6ms
Speed: 2.4ms preprocess, 207.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 210.5ms
Speed: 2.3ms preprocess, 210.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 196.3ms
Speed: 3.1ms preprocess, 196.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 194.9ms
Speed: 2.3ms preprocess, 194.9ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 210.8ms
Speed: 2.5ms preprocess, 210.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 215.1ms
Speed: 2.3ms preprocess, 215.1ms inference, 1.2ms postprocess per image a

INFO:werkzeug:127.0.0.1 - - [14/Jun/2024 05:26:04] "POST /process-video/ HTTP/1.1" 200 -



0: 480x640 1 person, 322.2ms
Speed: 3.6ms preprocess, 322.2ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 209.3ms
Speed: 2.5ms preprocess, 209.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 209.7ms
Speed: 2.3ms preprocess, 209.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 209.8ms
Speed: 2.0ms preprocess, 209.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 219.6ms
Speed: 5.3ms preprocess, 219.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 199.9ms
Speed: 2.1ms preprocess, 199.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 208.1ms
Speed: 2.2ms preprocess, 208.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 199.6ms
Speed: 2.2ms preprocess, 199.6ms inference, 1.3ms postprocess per image a

INFO:werkzeug:127.0.0.1 - - [14/Jun/2024 05:28:52] "POST /process-video/ HTTP/1.1" 200 -
